In [191]:
import requests
import json
import shapely
import copy
import numpy as np
from shapely.geometry import Point, Polygon


In [192]:
def get(name, url="http://51.250.19.71:8080/mmodel_attrs/"):
    r = requests.get(url+name)
    return json.dumps(r.json())

In [193]:
def np_to_shapely_polygons_z(arr, z=None):
    for a in arr:
        poly = []
        for pt in a:
            if z is not None:
                poly.append([pt[0], pt[1], z])
            else:
                poly.append([pt[0], pt[1], pt[2]])
                
        yield Polygon(poly)


        
def np_to_shapely_points(arr):
    for a in arr:
        yield Point(*a)

In [216]:
def extract_cord(arr, cord='z'):
    dict_ = {
        'x': 0, 
        'y': 1,
        'z': 2
    }
    crd = dict_[cord]
    vals=[]
    for a in arr:
        vals.append(np.unique(np.asarray(a)[...,crd])[0]) 
    return vals

def extract_unique_cord(arr, cord='z', **kwargs):
    return np.concatenate(np.asarray(extract_cord(arr, cord), dtype=object))

In [219]:
roofshapes = get('roofshapes')
zmask = get('zmask')
pointgrid=list(np_to_shapely_points(json.loads(roofshapes)['roofshapes']['pointgrid']))
low_poly = json.loads(zmask)['zmask']['low']
high_poly = json.loads(zmask)['zmask']['high']



In [224]:
def point_in_poly(low_poly=low_poly, high_poly=high_poly, pointgrid=pointgrid):
    point_pairs=[]
    low_polygons = list(np_to_shapely_polygons_z(low_poly, z = 0))
    high_polygons = list(np_to_shapely_polygons_z(high_poly, z = 0))
    z_low = extract_cord(low_poly)
    z_high = extract_cord(high_poly)
    
    for l in low_polygons:
        for h in high_polygons:
            for p in pointgrid:
                if l.contains(p) and h.contains(p):

                    p_low = [p.x, p.y, z_low[low_polygons.index(l)]]
                    p_high = [p.x, p.y, z_high[high_polygons.index(h)]]
                    point_pairs.append([p_low, p_high])
                else:
                    pass
    return point_pairs            

In [225]:
a=point_in_poly()
print(a)
    

[[[11850.000000000498, 4199.99999999727, 68677.31007990241], [11850.000000000498, 4199.99999999727, 74773.88555919548]], [[16050.000000000498, 4199.99999999727, 68677.31007990241], [16050.000000000498, 4199.99999999727, 74773.88555919548]], [[20250.0000000005, 4199.99999999727, 68677.31007990241], [20250.0000000005, 4199.99999999727, 74773.88555919548]], [[24450.0000000005, 4199.99999999727, 68677.31007990241], [24450.0000000005, 4199.99999999727, 72794.46644772356]], [[28650.0000000005, 4199.99999999727, 68677.31007990241], [28650.0000000005, 4199.99999999727, 72794.46644772356]], [[32850.000000000495, 4199.99999999727, 68677.31007990241], [32850.000000000495, 4199.99999999727, 72794.46644772356]], [[37050.000000000495, 4199.99999999727, 68677.31007990241], [37050.000000000495, 4199.99999999727, 72794.46644772356]], [[41250.000000000495, 4199.99999999727, 68677.31007990241], [41250.000000000495, 4199.99999999727, 72794.46644772356]], [[45450.000000000495, 4199.99999999727, 68677.31007